# Linear least squares

In [1]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2


In [2]:
from os.path import basename, exists


def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + local)


download("https://github.com/AllenDowney/ThinkStats/raw/v3/nb/thinkstats2.py")
download("https://github.com/AllenDowney/ThinkStats/raw/v3/nb/thinkplot.py")

In [3]:
import numpy as np
import random
import thinkstats2
import thinkplot

## Least squares fit

Correlation coefficients measure the strength and sign of a
relationship, but not the slope. There are several ways to estimate the
slope; the most common is a **linear least squares fit**. A "linear fit"
is a line intended to model the relationship between variables. A "least
squares" fit is one that minimizes the mean squared error (MSE) between
the line and the data.

Suppose we have a sequence of points, `ys`, that we want to express as a
function of another sequence `xs`. If there is a linear relationship
between `xs` and `ys` with intercept `inter` and slope `slope`, we
expect each `y[i]` to be `inter + slope * x[i]`.

But unless the correlation is perfect, this prediction is only
approximate. The vertical deviation from the line, or **residual**, is

The residuals might be due to random factors like measurement error, or
non-random factors that are unknown. For example, if we are trying to
predict weight as a function of height, unknown factors might include
diet, exercise, and body type.

If we get the parameters `inter` and `slope` wrong, the residuals get
bigger, so it makes intuitive sense that the parameters we want are the
ones that minimize the residuals.

We might try to minimize the absolute value of the residuals, or their
squares, or their cubes; but the most common choice is to minimize the
sum of squared residuals, `sum(res**2)`.

Why? There are three good reasons and one less important one:

-   Squaring has the feature of treating positive and negative residuals
    the same, which is usually what we want.

-   Squaring gives more weight to large residuals, but not so much
    weight that the largest residual always dominates.

-   If the residuals are uncorrelated and normally distributed with mean
    0 and constant (but unknown) variance, then the least squares fit is
    also the maximum likelihood estimator of `inter` and `slope`. See
    <https://en.wikipedia.org/wiki/Linear_regression>.

-   The values of `inter` and `slope` that minimize the squared
    residuals can be computed efficiently.

The last reason made sense when computational efficiency was more
important than choosing the method most appropriate to the problem at
hand. That's no longer the case, so it is worth considering whether
squared residuals are the right thing to minimize.

For example, if you are using `xs` to predict values of `ys`, guessing
too high might be better (or worse) than guessing too low. In that case
you might want to compute some cost function for each residual, and
minimize total cost, `sum(cost(res))`. However, computing a least
squares fit is quick, easy and often good enough.

## Implementation

`thinkstats2` provides simple functions that demonstrate linear least
squares:

In [5]:
from thinkstats2 import mean, mean_var, var, std, cov


def least_squares(xs, ys):
    meanx, varx = mean_var(xs)
    meany = mean(ys)
    slope = cov(xs, ys, meanx, meany) / varx
    inter = meany - slope * meanx
    return inter, slope

`LeastSquares` takes sequences `xs` and `ys` and returns the estimated
parameters `inter` and `slope`. For details on how it works, see
<http://wikipedia.org/wiki/Numerical_methods_for_linear_least_squares>.

`thinkstats2` also provides `FitLine`, which takes `inter` and `slope`
and returns the fitted line for a sequence of `xs`.

In [6]:
def fit_line(xs, inter, slope):
    fit_xs = np.sort(xs)
    fit_ys = inter + slope * fit_xs
    return fit_xs, fit_ys

We can use these functions to compute the least squares fit for birth
weight as a function of mother's age.

In [7]:
download("https://github.com/AllenDowney/ThinkStats/raw/v3/nb/nsfg.py")
download("https://github.com/AllenDowney/ThinkStats/raw/v3/nb/2002FemPreg.dct")
download(
    "https://github.com/AllenDowney/ThinkStats/raw/v3/nb/2002FemPreg.dat.gz"
)

In [8]:
import nsfg

live, firsts, others = nsfg.make_frames()
live = live.dropna(subset=["agepreg", "totalwgt_lb"])
ages = live.agepreg
weights = live.totalwgt_lb

In [9]:
inter, slope = thinkstats2.least_squares(ages, weights)
inter, slope

In [10]:
fit_xs, fit_ys = thinkstats2.fit_line(ages, inter, slope)

The estimated intercept and slope are 6.8 lbs and 0.017 lbs per year.
These values are hard to interpret in this form: the intercept is the
expected weight of a baby whose mother is 0 years old, which doesn't
make sense in context, and the slope is too small to grasp easily.

Instead of presenting the intercept at $x=0$, it is often helpful to
present the intercept at the mean of $x$. In this case the mean age is
about 25 years and the mean baby weight for a 25 year old mother is 7.3
pounds. The slope is 0.27 ounces per year, or 0.17 pounds per decade.

In [11]:
thinkplot.scatter(ages, weights, color="blue", alpha=0.1, s=10)
thinkplot.plot(fit_xs, fit_ys, color="white", linewidth=3)
thinkplot.plot(fit_xs, fit_ys, color="red", linewidth=2)
thinkplot.config(
    xlabel="Mother's age (years)",
    ylabel="Birth weight (lbs)",
    axis=[10, 45, 0, 15],
    legend=False,
)

Figure [\[linear1\]](#linear1){reference-type="ref" reference="linear1"}
shows a scatter plot of birth weight and age along with the fitted line.
It's a good idea to look at a figure like this to assess whether the
relationship is linear and whether the fitted line seems like a good
model of the relationship.

## Residuals

Another useful test is to plot the residuals. `thinkstats2` provides a
function that computes residuals:

In [12]:
def residuals(xs, ys, inter, slope):
    xs = np.asarray(xs)
    ys = np.asarray(ys)
    res = ys - (inter + slope * xs)
    return res

`Residuals` takes sequences `xs` and `ys` and estimated parameters
`inter` and `slope`. It returns the differences between the actual
values and the fitted line.

In [13]:
live["residual"] = residuals(ages, weights, inter, slope)

In [14]:
bins = np.arange(10, 48, 3)
indices = np.digitize(live.agepreg, bins)
groups = live.groupby(indices)
age_means = [group.agepreg.mean() for _, group in groups][1:-1]
age_means

In [15]:
cdfs = [thinkstats2.Cdf(group.residual) for _, group in groups][1:-1]

In [16]:
def plot_percentiles(age_means, cdfs):
    thinkplot.pre_plot(3)
    for percent in [75, 50, 25]:
        weight_percentiles = [cdf.percentile(percent) for cdf in cdfs]
        label = "%dth" % percent
        thinkplot.plot(age_means, weight_percentiles, label=label)

In [17]:
plot_percentiles(age_means, cdfs)
thinkplot.config(xlabel="Mother's age (years)", ylabel="Residual (lbs)", xlim=[10, 45])

To visualize the residuals, I group respondents by age and compute
percentiles in each group, as we saw in
Section [\[characterizing\]](#characterizing){reference-type="ref"
reference="characterizing"}.
Figure [\[linear2\]](#linear2){reference-type="ref" reference="linear2"}
shows the 25th, 50th and 75th percentiles of the residuals for each age
group. The median is near zero, as expected, and the interquartile range
is about 2 pounds. So if we know the mother's age, we can guess the
baby's weight within a pound, about 50% of the time.

Ideally these lines should be flat, indicating that the residuals are
random, and parallel, indicating that the variance of the residuals is
the same for all age groups. In fact, the lines are close to parallel,
so that's good; but they have some curvature, indicating that the
relationship is nonlinear. Nevertheless, the linear fit is a simple
model that is probably good enough for some purposes.

## Estimation

The parameters `slope` and `inter` are estimates based on a sample; like
other estimates, they are vulnerable to sampling bias, measurement
error, and sampling error. As discussed in
Chapter [\[estimation\]](#estimation){reference-type="ref"
reference="estimation"}, sampling bias is caused by non-representative
sampling, measurement error is caused by errors in collecting and
recording data, and sampling error is the result of measuring a sample
rather than the entire population.

To assess sampling error, we ask, "If we run this experiment again, how
much variability do we expect in the estimates?" We can answer this
question by running simulated experiments and computing sampling
distributions of the estimates.

I simulate the experiments by resampling the data; that is, I treat the
observed pregnancies as if they were the entire population and draw
samples, with replacement, from the observed sample.

In [18]:
def sample_rows(df, nrows, replace=False):
    """Choose a sample of rows from a DataFrame.

    df: DataFrame
    nrows: number of rows
    replace: whether to sample with replacement

    returns: DataDf
    """
    indices = np.random.choice(df.index, nrows, replace=replace)
    sample = df.loc[indices]
    return sample

In [19]:
def resample_rows(df):
    """Resamples rows from a DataFrame.

    df: DataFrame

    returns: DataFrame
    """
    return sample_rows(df, len(df), replace=True)

In [20]:
def sampling_distributions(live, iters=101):
    t = []
    for _ in range(iters):
        sample = thinkstats2.resample_rows(live)
        ages = sample.agepreg
        weights = sample.totalwgt_lb
        estimates = thinkstats2.least_squares(ages, weights)
        t.append(estimates)
    inters, slopes = zip(*t)
    return inters, slopes

In [21]:
inters, slopes = sampling_distributions(live, iters=1001)

`SamplingDistributions` takes a DataFrame with one row per live birth,
and `iters`, the number of experiments to simulate. It uses
`ResampleRows` to resample the observed pregnancies. We've already seen
`SampleRows`, which chooses random rows from a DataFrame. `thinkstats2`
also provides `ResampleRows`, which returns a sample the same size as
the original:

After resampling, we use the simulated sample to estimate parameters.
The result is two sequences: the estimated intercepts and estimated
slopes.

I summarize the sampling distributions by printing the standard error
and confidence interval:

In [26]:
def summarize(estimates, actual=None):
    mean = thinkstats2.mean(estimates)
    stderr = thinkstats2.std(estimates, mu=actual)
    cdf = thinkstats2.Cdf(estimates)
    ci = cdf.confidence_interval(90)
    print("mean, SE, CI", mean, stderr, ci)

`Summarize` takes a sequence of estimates and the actual value. It
prints the mean of the estimates, the standard error and a 90%
confidence interval.

In [27]:
summarize(inters)

In [28]:
summarize(slopes)

For the intercept, the mean estimate is 6.83, with standard error 0.07
and 90% confidence interval (6.71, 6.94). The estimated slope, in more
compact form, is 0.0174, SE 0.0028, CI (0.0126, 0.0220). There is almost
a factor of two between the low and high ends of this CI, so it should
be considered a rough estimate.

To visualize the sampling error of the estimate, we could plot all of
the fitted lines, or for a less cluttered representation, plot a 90%
confidence interval for each age. Here's the code:

In [29]:
for slope, inter in zip(slopes, inters):
    fxs, fys = fit_line(age_means, inter, slope)
    thinkplot.plot(fxs, fys, color="gray", alpha=0.01)
thinkplot.config(xlabel="Mother's age (years)", ylabel="Residual (lbs)", xlim=[10, 45])

In [30]:
def plot_confidence_intervals(xs, inters, slopes, percent=90, **options):
    fys_seq = []
    for inter, slope in zip(inters, slopes):
        fxs, fys = thinkstats2.fit_line(xs, inter, slope)
        fys_seq.append(fys)
    p = (100 - percent) / 2
    percents = p, 100 - p
    low, high = thinkstats2.percentile_rows(fys_seq, percents)
    thinkplot.fill_between(fxs, low, high, **options)

`xs` is the sequence of mother's age. `inters` and `slopes` are the
estimated parameters generated by `SamplingDistributions`. `percent`
indicates which confidence interval to plot.

`PlotConfidenceIntervals` generates a fitted line for each pair of
`inter` and `slope` and stores the results in a sequence, `fys_seq`.
Then it uses `PercentileRows` to select the upper and lower percentiles
of `y` for each value of `x`. For a 90% confidence interval, it selects
the 5th and 95th percentiles. `FillBetween` draws a polygon that fills
the space between two lines.

In [31]:
plot_confidence_intervals(
    age_means, inters, slopes, percent=90, color="gray", alpha=0.3, label="90% CI"
)
plot_confidence_intervals(
    age_means, inters, slopes, percent=50, color="gray", alpha=0.5, label="50% CI"
)
thinkplot.config(xlabel="Mother's age (years)", ylabel="Residual (lbs)", xlim=[10, 45])

Figure [\[linear3\]](#linear3){reference-type="ref" reference="linear3"}
shows the 50% and 90% confidence intervals for curves fitted to birth
weight as a function of mother's age. The vertical width of the region
represents the effect of sampling error; the effect is smaller for
values near the mean and larger for the extremes.

## Goodness of fit

There are several ways to measure the quality of a linear model, or
**goodness of fit**. One of the simplest is the standard deviation of
the residuals.

If you use a linear model to make predictions, `Std(res)` is the root
mean squared error (RMSE) of your predictions. For example, if you use
mother's age to guess birth weight, the RMSE of your guess would be 1.40
lbs.

If you guess birth weight without knowing the mother's age, the RMSE of
your guess is `Std(ys)`, which is 1.41 lbs. So in this example, knowing
a mother's age does not improve the predictions substantially.

Another way to measure goodness of fit is the **coefficient of
determination**, usually denoted $R^2$ and called "R-squared":

In [32]:
def coef_determination(ys, res):
    return 1 - var(res) / var(ys)

In [33]:
inter, slope = least_squares(ages, weights)
res = residuals(ages, weights, inter, slope)
r2 = coef_determination(weights, res)
r2

In [34]:
print("rho", thinkstats2.corr(ages, weights))
print("R", np.sqrt(r2))

In [35]:
print("Std(ys)", std(weights))
print("Std(res)", std(res))

`Var(res)` is the MSE of your guesses using the model, `Var(ys)` is the
MSE without it. So their ratio is the fraction of MSE that remains if
you use the model, and $R^2$ is the fraction of MSE the model
eliminates.

For birth weight and mother's age, $R^2$ is 0.0047, which means that
mother's age predicts about half of 1% of variance in birth weight.

There is a simple relationship between the coefficient of determination
and Pearson's coefficient of correlation: $R^2 = \rho^2$. For example,
if $\rho$ is 0.8 or -0.8, $R^2 = 0.64$.

Although $\rho$ and $R^2$ are often used to quantify the strength of a
relationship, they are not easy to interpret in terms of predictive
power. In my opinion, `Std(res)` is the best representation of the
quality of prediction, especially if it is presented in relation to
`Std(ys)`.

For example, when people talk about the validity of the SAT (a
standardized test used for college admission in the U.S.) they often
talk about correlations between SAT scores and other measures of
intelligence.

According to one study, there is a Pearson correlation of $\rho=0.72$
between total SAT scores and IQ scores, which sounds like a strong
correlation. But $R^2 = \rho^2 = 0.52$, so SAT scores account for only
52% of variance in IQ.

IQ scores are normalized with `Std(ys) = 15`, so

In [36]:
var_ys = 15**2
rho = 0.72
r2 = rho**2
var_res = (1 - r2) * var_ys
std_res = np.sqrt(var_res)
std_res

So using SAT score to predict IQ reduces RMSE from 15 points to 10.4
points. A correlation of 0.72 yields a reduction in RMSE of only 31%.

If you see a correlation that looks impressive, remember that $R^2$ is a
better indicator of reduction in MSE, and reduction in RMSE is a better
indicator of predictive power.

## Testing a linear model

The effect of mother's age on birth weight is small, and has little
predictive power. So is it possible that the apparent relationship is
due to chance? There are several ways we might test the results of a
linear fit.

One option is to test whether the apparent reduction in MSE is due to
chance. In that case, the test statistic is $R^2$ and the null
hypothesis is that there is no relationship between the variables. We
can simulate the null hypothesis by permutation, as in
Section [\[corrtest\]](#corrtest){reference-type="ref"
reference="corrtest"}, when we tested the correlation between mother's
age and birth weight. In fact, because $R^2 = \rho^2$, a one-sided test
of $R^2$ is equivalent to a two-sided test of $\rho$. We've already done
that test, and found $p < 0.001$, so we conclude that the apparent
relationship between mother's age and birth weight is statistically
significant.

Another approach is to test whether the apparent slope is due to chance.
The null hypothesis is that the slope is actually zero; in that case we
can model the birth weights as random variations around their mean.
Here's a HypothesisTest for this model:

In [37]:
class SlopeTest(thinkstats2.HypothesisTest):

    def test_statistic(self, data):
        ages, weights = data
        _, slope = thinkstats2.least_squares(ages, weights)
        return slope

    def make_model(self):
        _, weights = self.data
        self.ybar = weights.mean()
        self.res = weights - self.ybar

    def run_model(self):
        ages, _ = self.data
        weights = self.ybar + np.random.permutation(self.res)
        return ages, weights

The data are represented as sequences of ages and weights. The test
statistic is the slope estimated by `LeastSquares`. The model of the
null hypothesis is represented by the mean weight of all babies and the
deviations from the mean. To generate simulated data, we permute the
deviations and add them to the mean.

Here's the code that runs the hypothesis test:

In [38]:
live = live.dropna(subset=["agepreg", "totalwgt_lb"])
ht = SlopeTest((live.agepreg, live.totalwgt_lb))
p_value = ht.p_value()
p_value

In [39]:
ht.actual, ht.max_test_stat()

The p-value is less than $0.001$, so although the estimated slope is
small, it is unlikely to be due to chance.

Estimating the p-value by simulating the null hypothesis is strictly
correct, but there is a simpler alternative. Remember that we already
computed the sampling distribution of the slope, in
Section [\[regest\]](#regest){reference-type="ref" reference="regest"}.
To do that, we assumed that the observed slope was correct and simulated
experiments by resampling.

Figure [\[linear4\]](#linear4){reference-type="ref" reference="linear4"}
shows the sampling distribution of the slope, from
Section [\[regest\]](#regest){reference-type="ref" reference="regest"},
and the distribution of slopes generated under the null hypothesis. The
sampling distribution is centered about the estimated slope, 0.017
lbs/year, and the slopes under the null hypothesis are centered around
0; but other than that, the distributions are identical. The
distributions are also symmetric, for reasons we will see in
Section [\[CLT\]](#CLT){reference-type="ref" reference="CLT"}.

In [40]:
sampling_cdf = thinkstats2.Cdf(slopes)
thinkplot.pre_plot(2)
thinkplot.plot([0, 0], [0, 1], color="0.8")
ht.plot_cdf(label="null hypothesis")
thinkplot.cdf(sampling_cdf, label="sampling distribution")
thinkplot.config(
    xlabel="slope (lbs / year)",
    ylabel="CDF",
    xlim=[-0.03, 0.03],
    legend=True,
    loc="upper left",
)

So we could estimate the p-value two ways:

-   Compute the probability that the slope under the null hypothesis
    exceeds the observed slope.

-   Compute the probability that the slope in the sampling distribution
    falls below 0. (If the estimated slope were negative, we would
    compute the probability that the slope in the sampling distribution
    exceeds 0.)

The second option is easier because we normally want to compute the
sampling distribution of the parameters anyway. And it is a good
approximation unless the sample size is small *and* the distribution of
residuals is skewed. Even then, it is usually good enough, because
p-values don't have to be precise.

Here's the code that estimates the p-value of the slope using the
sampling distribution:

In [41]:
inters, slopes = sampling_distributions(live, iters=1001)
slope_cdf = thinkstats2.Cdf(slopes)
p_value = slope_cdf[0]
p_value

Again, we find $p < 0.001$.

## Weighted resampling

So far we have treated the NSFG data as if it were a representative
sample, but as I mentioned in
Section [\[nsfg\]](#nsfg){reference-type="ref" reference="nsfg"}, it is
not. The survey deliberately oversamples several groups in order to
improve the chance of getting statistically significant results; that
is, in order to improve the power of tests involving these groups.

This survey design is useful for many purposes, but it means that we
cannot use the sample to estimate values for the general population
without accounting for the sampling process.

For each respondent, the NSFG data includes a variable called
`finalwgt`, which is the number of people in the general population the
respondent represents. This value is called a **sampling weight**, or
just "weight."

As an example, if you survey 100,000 people in a country of 300 million,
each respondent represents 3,000 people. If you oversample one group by
a factor of 2, each person in the oversampled group would have a lower
weight, about 1500.

To correct for oversampling, we can use resampling; that is, we can draw
samples from the survey using probabilities proportional to sampling
weights. Then, for any quantity we want to estimate, we can generate
sampling distributions, standard errors, and confidence intervals. As an
example, I will estimate mean birth weight with and without sampling
weights.

In Section [\[regest\]](#regest){reference-type="ref"
reference="regest"}, we saw `ResampleRows`, which chooses rows from a
DataFrame, giving each row the same probability. Now we need to do the
same thing using probabilities proportional to sampling weights.
`ResampleRowsWeighted` takes a DataFrame, resamples rows according to
the weights in `finalwgt`, and returns a DataFrame containing the
resampled rows:

In [42]:
def resample_rows_weighted(df, column="finalwgt"):
    weights = df[column]
    cdf = thinkstats2.Cdf(dict(weights))
    indices = cdf.sample(len(weights))
    sample = df.loc[indices]
    return sample

`weights` is a Series; converting it to a dictionary makes a map from
the indices to the weights. In `cdf` the values are indices and the
probabilities are proportional to the weights.

`indices` is a sequence of row indices; `sample` is a DataFrame that
contains the selected rows. Since we sample with replacement, the same
row might appear more than once.

Now we can compare the effect of resampling with and without weights.
Without weights, we generate the sampling distribution like this:

In [43]:
iters = 100
estimates = [resample_rows_weighted(live).totalwgt_lb.mean() for _ in range(iters)]
summarize(estimates)

With weights, it looks like this:

In [ ]:
estimates = [thinkstats2.resample_rows(live).totalwgt_lb.mean() for _ in
    range(iters)]
summarize(estimates)

The following table summarizes the results:

 ```
  ------------ -------------- ---------- --------------
                 mean birth    standard      90% CI
                weight (lbs)    error    
  Unweighted        7.27        0.014     (7.24, 7.29)
  Weighted          7.35        0.014     (7.32, 7.37)
  ------------ -------------- ---------- --------------
```

In this example, the effect of weighting is small but non-negligible.
The difference in estimated means, with and without weighting, is about
0.08 pounds, or 1.3 ounces. This difference is substantially larger than
the standard error of the estimate, 0.014 pounds, which implies that the
difference is not due to chance.

## Glossary

-   **linear fit**: a line intended to model the relationship between
    variables.

-   **least squares fit**: A model of a dataset that minimizes the sum
    of squares of the residuals.

-   **residual**: The deviation of an actual value from a model.

-   **goodness of fit**: A measure of how well a model fits data.

-   **coefficient of determination**: A statistic intended to quantify
    goodness of fit.

-   **sampling weight**: A value associated with an observation in a
    sample that indicates what part of the population it represents.

# Exercises

**Exercise:** Use `ResampleRows` and generate a list of estimates for the mean birth weight.  Use `Summarize` to compute the SE and CI for these estimates.

In [44]:
iters = 1000
estimates = [resample_rows(live).totalwgt_lb.mean() for _ in range(iters)]
summarize(estimates)

The difference is non-negligible, which suggests that there are differences in birth weight between the strata in the survey.

**Exercise:** Using the data from the BRFSS, compute the linear least squares fit for log(weight) versus height. How would you best present the estimated parameters for a model like this where one of the variables is log-transformed? If you were trying to guess someone’s weight, how much would it help to know their height?

Like the NSFG, the BRFSS oversamples some groups and provides a sampling weight for each respondent. In the BRFSS data, the variable name for these weights is totalwt. Use resampling, with and without weights, to estimate the mean height of respondents in the BRFSS, the standard error of the mean, and a 90% confidence interval. How much does correct weighting affect the estimates?

Read the BRFSS data and extract heights and log weights.

In [45]:
download("https://github.com/AllenDowney/ThinkStats/raw/v3/nb/brfss.py")
download("https://github.com/AllenDowney/ThinkStats/raw/v3/nb/CDBRFS08.ASC.gz")

In [46]:
import brfss

df = brfss.read_brfss(nrows=None)
df = df.dropna(subset=["htm3", "wtkg2"])
heights, weights = df.htm3, df.wtkg2
log_weights = np.log10(weights)

Estimate intercept and slope.

In [47]:
inter, slope = thinkstats2.least_squares(heights, log_weights)
inter, slope

Make a scatter plot of the data and show the fitted line.

In [48]:
thinkplot.scatter(heights, log_weights, alpha=0.01, s=5)
fxs, fys = thinkstats2.fit_line(heights, inter, slope)
thinkplot.plot(fxs, fys, color="red")
thinkplot.config(xlabel="Height (cm)", ylabel="log10 weight (kg)", legend=False)

Make the same plot but apply the inverse transform to show weights on a linear (not log) scale.

In [49]:
thinkplot.scatter(heights, weights, alpha=0.01, s=5)
fxs, fys = thinkstats2.fit_line(heights, inter, slope)
thinkplot.plot(fxs, 10**fys, color="red")
thinkplot.config(xlabel="Height (cm)", ylabel="Weight (kg)", legend=False)

Plot percentiles of the residuals.

In [50]:
res = thinkstats2.residuals(heights, log_weights, inter, slope)
df["residual"] = res
bins = np.arange(130, 210, 5)
indices = np.digitize(df.htm3, bins)
groups = df.groupby(indices)
means = [group.htm3.mean() for i, group in groups][1:-1]
cdfs = [thinkstats2.Cdf(group.residual) for i, group in groups][1:-1]
thinkplot.pre_plot(3)
for percent in [75, 50, 25]:
    ys = [cdf.percentile(percent) for cdf in cdfs]
    label = "%dth" % percent
    thinkplot.plot(means, ys, label=label)
thinkplot.config(xlabel="height (cm)", ylabel="residual weight (kg)", legend=False)

Compute correlation.

In [51]:
rho = thinkstats2.corr(heights, log_weights)
rho

Compute coefficient of determination.

In [52]:
r2 = thinkstats2.coef_determination(log_weights, res)
r2

Confirm that $R^2 = \rho^2$.

In [53]:
np.isclose(rho**2, r2)

Compute Std(ys), which is the RMSE of predictions that don't use height.

In [54]:
std_ys = thinkstats2.std(log_weights)
std_ys

Compute Std(res), the RMSE of predictions that do use height.

In [55]:
std_res = thinkstats2.std(res)
std_res

How much does height information reduce RMSE?

In [56]:
1 - std_res / std_ys

Use resampling to compute sampling distributions for inter and slope.

In [57]:
t = []
for _ in range(100):
    sample = thinkstats2.resample_rows(df)
    estimates = thinkstats2.least_squares(sample.htm3, np.log10(sample.wtkg2))
    t.append(estimates)
inters, slopes = zip(*t)

Plot the sampling distribution of slope.

In [58]:
cdf = thinkstats2.Cdf(slopes)
thinkplot.cdf(cdf)

Compute the p-value of the slope.

In [59]:
pvalue = cdf[0]
pvalue

Compute the 90% confidence interval of slope.

In [60]:
ci = cdf.percentile(5), cdf.percentile(95)
ci

Compute the mean of the sampling distribution.

In [61]:
mean = thinkstats2.mean(slopes)
mean

Compute the standard deviation of the sampling distribution, which is the standard error.

In [62]:
stderr = thinkstats2.std(slopes)
stderr

Resample rows without weights, compute mean height, and summarize results.

In [63]:
estimates_unweighted = [thinkstats2.resample_rows(df).htm3.mean() for _ in range(100)]
summarize(estimates_unweighted)

Resample rows with weights.  Note that the weight column in this dataset is called `finalwt`.

In [65]:
estimates_weighted = [
    resample_rows_weighted(df, "finalwt").htm3.mean() for _ in range(100)
]
summarize(estimates_weighted)